In [ ]:
import pandas as pd
RAW_PATH = "dgbbkiqniqb4dpwh.csv"

OUT_TRADE_CLEAN = "trace_cleaned_stage1.csv"
OUT_ISSUER_DAILY_LONG = "cleaned_issuer_daily_long.csv"
OUT_PANEL_WIDE = "preliminary_issuer_panel.csv"
OUT_VALID_ISSUERS = "valid_issuers_list.csv"

MIN_TRADING_DAYS_PER_ISSUER = 60

df = pd.read_csv(RAW_PATH, low_memory=False)

required_cols = [
    "sub_prd_type", "trc_st", "wis_fl", "asof_cd", "sale_cndtn_cd",
    "yld_pt", "trd_exctn_dt", "company_symbol", "cusip_id"
]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}\nAvailable cols: {df.columns.tolist()}")

clean = df.copy()

# Corporate bonds only
clean = clean[clean["sub_prd_type"] == "CORP"]

# Matched trades only
clean = clean[clean["trc_st"] == "M"]

# Drop when-issued trades
clean = clean[clean["wis_fl"] != "Y"]

# Drop as-of trades
clean = clean[clean["asof_cd"] != "Y"]

# Drop special-condition trades (keep regular conditions)
sale = clean["sale_cndtn_cd"]
clean = clean[sale.isna() | (sale.astype(str).str.strip() == "")]

# Keep valid numeric yields
clean["yld_pt"] = pd.to_numeric(clean["yld_pt"], errors="coerce")
clean = clean[clean["yld_pt"].notna()]

# Parse trade date
clean["trd_exctn_dt"] = pd.to_datetime(clean["trd_exctn_dt"], errors="coerce")
clean = clean[clean["trd_exctn_dt"].notna()]

clean = clean[clean["company_symbol"].notna()]
clean = clean[clean["cusip_id"].notna()]

# Save trade-level cleaned file
clean.to_csv(OUT_TRADE_CLEAN, index=False)
print(f"Saved trade-level cleaned file: {OUT_TRADE_CLEAN}  | rows={len(clean):,}")

issuer_daily = (
    clean.groupby(["trd_exctn_dt", "company_symbol"])["yld_pt"]
    .median()
    .reset_index()
)

# Keep issuers with enough time coverage
issuer_counts = issuer_daily.groupby("company_symbol")["trd_exctn_dt"].nunique()
valid_issuers = issuer_counts[issuer_counts >= MIN_TRADING_DAYS_PER_ISSUER].index.tolist()

issuer_daily_filtered = issuer_daily[issuer_daily["company_symbol"].isin(valid_issuers)].copy()

issuer_daily_filtered.to_csv(OUT_ISSUER_DAILY_LONG, index=False)
pd.DataFrame({"company_symbol": valid_issuers}).to_csv(OUT_VALID_ISSUERS, index=False)

print(f"Saved issuer daily (long): {OUT_ISSUER_DAILY_LONG}  | rows={len(issuer_daily_filtered):,}")
print(f"Saved valid issuer list:   {OUT_VALID_ISSUERS}      | issuers={len(valid_issuers):,}")

panel = issuer_daily_filtered.pivot(
    index="trd_exctn_dt",
    columns="company_symbol",
    values="yld_pt"
).sort_index()

panel.to_csv(OUT_PANEL_WIDE)
print(f"Saved panel (wide):        {OUT_PANEL_WIDE}         | shape={panel.shape}")

Saved trade-level cleaned file: trace_cleaned_stage1.csv  | rows=891,442
Saved issuer daily (long): cleaned_issuer_daily_long.csv  | rows=1,908
Saved valid issuer list:   valid_issuers_list.csv      | issuers=25
Saved panel (wide):        preliminary_issuer_panel.csv         | shape=(93, 25)
